# Laboratorium 3
#### Bartosz Hanc

Zadanie dotyczy wykorzystania odległości edycyjnej.

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie
   przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w
   minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez
   wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana.
   "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być
   zrealizowana następująco:
    1. *k*los (dodanie litery k)
    2. klo*c* (zamiana s->c) 

3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
    * los - kloc
    * Łódź - Lodz
    * kwintesencja - quintessence
    * ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG

4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału
   załączonego tekstu na tokeny.

6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.

8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff,
   tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć
   się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia
   pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.

9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków
   przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

---

In [1]:
from enum import Enum


def Levenstine_dist(s1: str, s2: str) -> int:
    """
    Returns Levenstein distance between words s1 and s2.
    """

    m, n = len(s1), len(s2)
    s1 = " " + s1
    s2 = " " + s2
    d = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    parent = [[None for _ in range(n + 1)] for _ in range(m + 1)]
    operation = [[None for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        d[i][0] = i
        if i > 0:
            operation[i][0] = ("delete", i - 1)

    for j in range(n + 1):
        d[0][j] = j
        if j > 0:
            operation[0][j] = ("insert", -1, s2[j])

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if s1[i] == s2[j] else 1
            min_cost = min(
                d[i - 1][j] + 1,  # deletion
                d[i][j - 1] + 1,  # insertion
                d[i - 1][j - 1] + cost,  # change
            )

            if min_cost == d[i - 1][j] + 1:
                operation[i][j] = ("delete", i - 1)
                parent[i][j] = (i - 1, j)

            elif min_cost == d[i][j - 1] + 1:
                operation[i][j] = ("insert", i - 1, s2[j])
                parent[i][j] = (i, j - 1)

            elif min_cost == d[i - 1][j - 1] + 1:
                operation[i][j] = ("change", i - 1, s2[j])
                parent[i][j] = (i - 1, j - 1)

            else:
                parent[i][j] = (i - 1, j - 1)

            d[i][j] = min_cost

    steps = []
    i, j = m, n
    while parent[i][j] != None:
        if operation[i][j] != None:
            steps.append(operation[i][j])
        i, j = parent[i][j]
    if operation[i][j] != None:
        steps.append(operation[i][j])

    return d[m][n], steps[::-1]


def visualize(s1: str, s2: str) -> None:
    _, steps = Levenstine_dist(s1, s2)
    s, di = s1, 0
    print(s1)
    for step in steps:
        match step[0]:
            case "delete":
                i = step[1]
                print(f"{s[:i+di]}*{s[i+di]}*{s[i+di+1:]} (delete {s[i+di]})")
                s = s[: i + di] + s[i + di + 1 :]
                di -= 1

            case "insert":
                i, char = step[1], step[2]
                print(f"{s[:i+di+1]}*{char}*{s[i+di+1:]} (insert {char})")
                s = s[: i + di + 1] + char + s[i + di + 1 :]
                di += 1

            case "change":
                i, char = step[1], step[2]
                print(f"{s[:i+di]}*{char}*{s[i+di+1:]} (change {s[i+di]}->{char})")
                s = s[: i + di] + char + s[i + di + 1 :]

    print(f"{s} == {s2} ({s == s2})")


In [2]:
for s1, s2 in (
    ("los", "kloc"),
    ("Łódź", "Lodz"),
    ("kwintesencja", "quintessence"),
    ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")
):
    visualize(s1, s2)
    print("-------------------------")


los
*k*los (insert k)
klo*c* (change s->c)
kloc == kloc (True)
-------------------------
Łódź
*L*ódź (change Ł->L)
L*o*dź (change ó->o)
Lod*z* (change ź->z)
Lodz == Lodz (True)
-------------------------
kwintesencja
*q*wintesencja (change k->q)
q*u*intesencja (change w->u)
quintes*s*encja (insert s)
quintessenc*e*a (change j->e)
quintessence*a* (delete a)
quintessence == quintessence (True)
-------------------------
ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG (change A->G)
ATGAG*G*CTTACCGCCTCG (change T->G)
ATGAGGCT*C*TACCGCCTCG (insert C)
ATGAGGCTCT*G*CCGCCTCG (change A->G)
ATGAGGCTCTG*G*CCGCCTCG (insert G)
ATGAGGCTCTGGCC*G*CCTCG (delete G)
ATGAGGCTCTGGCCCCT*C*G (delete C)
ATGAGGCTCTGGCCCCTG == ATGAGGCTCTGGCCCCTG (True)
-------------------------


In [3]:
def lcs(A, B):
    n, m = len(A), len(B)
    C = [[None for _ in range(m + 1)] for _ in range(n + 1)]

    for i in range(n + 1):
        C[i][0] = 0

    for i in range(m + 1):
        C[0][i] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if A[i - 1] == B[j - 1]:
                C[i][j] = C[i - 1][j - 1] + 1
            else:
                C[i][j] = max(C[i - 1][j], C[i][j - 1])

    res = []
    i, j = n, m
    while (i, j) != (0, 0):
        if C[i - 1][j] == C[i][j]:
            i, j = i - 1, j
        elif C[i][j - 1] == C[i][j]:
            i, j = i, j - 1
        else:
            res.append(A[i - 1])
            i, j = i - 1, j - 1
            
    return res[::-1]


In [4]:
import spacy
import random

nlp = spacy.blank("pl")
with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as file:
    #text = file.read()
    tokens = nlp(file.read())

text1, text2 = "", ""
tok_rand1 = set(random.choices(list(tokens), k=int(0.97 * len(tokens))))
tok_rand2 = set(random.choices(list(tokens), k=int(0.97 * len(tokens))))
for token in tokens:
    if token not in tok_rand1:
        text1 += token.text
    if token not in tok_rand2:
        text2 += token.text

lcs(tokens, tokens)


[William,
 Shakespeare,
 
 ,
 Romeo,
 i,
 Julia,
 ,
 tłum,
 .,
 Józef,
 Paszkowski,
 
 ,
 ISBN,
 978,
 -,
 83,
 -,
 288,
 -,
 2903,
 -,
 9,
 
 
 
 ,
 OSOBY,
 :,
 
  ,
 *,
 ESKALUS,
 —,
 książę,
 panujący,
 w,
 Weronie,
 
  ,
 *,
 PARYS,
 —,
 młody,
 Weroneńczyk,
 szlachetnego,
 rodu,
 ,,
 krewny,
 księcia,
 
  ,
 *,
 MONTEKI,
 ,,
 KAPULET,
 —,
 naczelnicy,
 dwóch,
 domów,
 nieprzyjaznych,
 sobie,
 
  ,
 *,
 STARZEC,
 —,
 stryjeczny,
 brat,
 Kapuleta,
 
  ,
 *,
 ROMEO,
 —,
 syn,
 Montekiego,
 
  ,
 *,
 MERKUCJO,
 —,
 krewny,
 księcia,
 
  ,
 *,
 BENWOLIO,
 —,
 synowiec,
 Montekiego,
 
  ,
 *,
 TYBALT,
 —,
 krewny,
 Pani,
 Kapulet,
 
  ,
 *,
 LAURENTY,
 —,
 ojciec,
 franciszkanin,
 
  ,
 *,
 JAN,
 —,
 brat,
 z,
 tegoż,
 zgromadzenia,
 
  ,
 *,
 BALTAZAR,
 —,
 służący,
 Romea,
 
  ,
 *,
 SAMSON,
 ,,
 GRZEGORZ,
 —,
 słudzy,
 Kapuleta,
 
  ,
 *,
 ABRAHAM,
 —,
 służący,
 Montekiego,
 
  ,
 *,
 APTEKARZ,
 
  ,
 *,
 TRZECH,
 MUZYKANTÓW,
 
  ,
 *,
 PAŹ,
 PARYSA,
 
  ,
 *,
 PIOTR,
 
  ,
 *,
 DOW